## 텍스트 분석
* 텍스트 분류 : 지도학습
* 감성분석    : 지도, 비지도
* 텍스트 요약 : 토픽 모델링
* 텍스트 군집화와 유사도 측정 : 텍스트분류를 비지도학습으로

## 픽쳐 벡터화 또는 피처 추출
* BOW (Bag of Words)
* Word2Vec

## 텍스트 분석 수행 프로세스
1. 텍스트 사전 준비 (정규화)
    * 클렌징(Cleansing) 작업
        - 대/소문자 변경, 특수문자 삭제 등
        - HTML, XML 태그나 특정기호 등 사전제거
    * 단어(Word) 등의 토큰화(Tokenization) 작업 : API 제공
        - 문서에서 문장을 분리하는 문장 토근화 : 마침표(.), 개행문자 등 문장의 마지막을 뜻하는 기호로 분리, 정규표현식 가능
        - 문장에서 단어를 분리하는 단어 토근화 : 공백,콤마(,), 마침표(.), 개행문자 등으로 단어분리, 정규표현식 가능
    * 필터링/의미없는 단어(Stop word) 제거 /철자 수정
    * 어근 추출(Stemming/Lemmatization) 
2. 피처 벡터화/추출
    * 피처 추출하고 벡터값을 할당
    * BOW : count기반 벡터화, TF-IDF기반 벡터화
3. ML 모델 수립 및 학습/예측/평가

In [ ]:
## 파이썬 기반 NLP, 텍스트 분석 전용 패키지
    * NLTK : 거의 모든 영역 커버, 속도 아쉬움 대량 데이터에서 제대로 활용 X
    * Genism : 토픽 모델링 분야, Word2Vec 구현 등 신기능 제공
    * SpaCy : 최근 가장 주목받는 NLP패키지
### 사이키런에서 제공하는 라이브러리 있음. 전용패키지와 결합해 애플작성이 많음

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
# 문장 토큰화(text tokenization) : 문장의 마침표, 개행문자 등 문장의 마지막을 뜻하는 기호로 분리 또한 정규표현식 가능
from nltk import sent_tokenize
text_sample = 'The Matrix is everywhere its all around us, here even in this room. \
               You can see it out your window or on your television. \
               You feel it when you go to work, or go to church or pay your taxes.'
sentences = sent_tokenize(text=text_sample)
print(type(sentences),len(sentences))
print(sentences)



<class 'list'> 3
['The Matrix is everywhere its all around us, here even in this room.', 'You can see it out your window or on your television.', 'You feel it when you go to work, or go to church or pay your taxes.']


In [6]:
# 단어 토큰화(word tokenization) : 공백,콤마(,), 마침표(.), 개행문자 등으로 단어분리, 정규표현식 가능
from nltk import word_tokenize

sentence = "The Matrix is everywhere its all around us, here even in this room."
words = word_tokenize(sentence)
print(type(words), len(words))
print(words)

<class 'list'> 15
['The', 'Matrix', 'is', 'everywhere', 'its', 'all', 'around', 'us', ',', 'here', 'even', 'in', 'this', 'room', '.']


In [7]:
from nltk import word_tokenize, sent_tokenize

#여러개의 문장으로 된 입력 데이터를 문장별로 단어 토큰화 만드는 함수 생성
def tokenize_text(text):
    
    # 문장별로 분리 토큰
    sentences = sent_tokenize(text)
    # 분리된 문장별 단어 토큰화
    word_tokens = [word_tokenize(sentence) for sentence in sentences]
    return word_tokens

#여러 문장들에 대해 문장별 단어 토큰화 수행. 
word_tokens = tokenize_text(text_sample)
print(type(word_tokens),len(word_tokens))
print(word_tokens)

<class 'list'> 3
[['The', 'Matrix', 'is', 'everywhere', 'its', 'all', 'around', 'us', ',', 'here', 'even', 'in', 'this', 'room', '.'], ['You', 'can', 'see', 'it', 'out', 'your', 'window', 'or', 'on', 'your', 'television', '.'], ['You', 'feel', 'it', 'when', 'you', 'go', 'to', 'work', ',', 'or', 'go', 'to', 'church', 'or', 'pay', 'your', 'taxes', '.']]


In [8]:
### 문맥적인 의미를 해결해 보고자 도입된 것이 n-gram 

In [9]:
# Stopwords(문맥적으로 의미없이 빈번하게 나타나는) 제거 => 언어별 제공됨
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
print('영어 stop words 갯수:',len(nltk.corpus.stopwords.words('english')))
print(nltk.corpus.stopwords.words('english')[:20])

영어 stop words 갯수: 179
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his']


In [12]:
# (문장 토큰화 -> 단어 토근화) -> Stopwords 제거 
import nltk

stopwords = nltk.corpus.stopwords.words('english')
all_tokens = []
# 위 예제의 3개의 문장별로 얻은 word_tokens list 에 대해 stop word 제거 Loop
for sentence in word_tokens:
    filtered_words=[]
    # 개별 문장별로 tokenize된 sentence list에 대해 stop word 제거 Loop
    for word in sentence:
        #소문자로 모두 변환합니다. 
        word = word.lower()
        # tokenize 된 개별 word가 stop words 들의 단어에 포함되지 않으면 word_tokens에 추가
        if word not in stopwords:
            filtered_words.append(word)
    all_tokens.append(filtered_words)
    
print(all_tokens)

[['matrix', 'everywhere', 'around', 'us', ',', 'even', 'room', '.'], ['see', 'window', 'television', '.'], ['feel', 'go', 'work', ',', 'go', 'church', 'pay', 'taxes', '.']]


## Stemming, Lemmatization : 문법적 또는 의미적으로 변화하는 단어의 원형을 찾는 것
    * 영어의 경우 과서/현재, 3인칭 단수여부, 진행형, 비교/최상의 형용사의 변화
    * Stemming : 
        - Porter, Lancaster, Snowball Stemmer
    * Lemmatization : 품사와 같은 문법적인 요소와  더 의미적인 부분을 감안해 정확한 철자로 된 어근 단어 찾아줌
        - WordNetLemmatizer

In [14]:
from nltk.stem import LancasterStemmer
stemmer = LancasterStemmer()

print(stemmer.stem('working'),stemmer.stem('works'),stemmer.stem('worked'))
print(stemmer.stem('amusing'),stemmer.stem('amuses'),stemmer.stem('amused')) # amuse 정답
print(stemmer.stem('happier'),stemmer.stem('happiest')) # happy
print(stemmer.stem('fancier'),stemmer.stem('fanciest')) # fancy

work work work
amus amus amus
happy happiest
fant fanciest


In [16]:
  import nltk
  nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [17]:
from nltk.stem import WordNetLemmatizer

lemma = WordNetLemmatizer()
print(lemma.lemmatize('amusing','v'),lemma.lemmatize('amuses','v'),lemma.lemmatize('amused','v')) # 동사 v
print(lemma.lemmatize('happier','a'),lemma.lemmatize('happiest','a')) # 형용사 a 
print(lemma.lemmatize('fancier','a'),lemma.lemmatize('fanciest','a'))

amuse amuse amuse
happy happy
fancy fancy
